In [1]:
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from photutils.segmentation import detect_sources
import matplotlib.pyplot as plt
from photutils.background import Background2D
from photutils.segmentation import SourceCatalog
from photutils.background import StdBackgroundRMS
from photutils.segmentation import detect_sources, SegmentationImage
from astropy.stats import SigmaClip
from photutils.segmentation import deblend_sources


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/nasekhian/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/xm/sl735y1n4m30kfvcxl7h4p6w0000gn/T/ipykernel_25321/3016963944.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Define the data folder
data_folder = '/Users/nasekhian/Documents/GitHub/IMcascade'
num_simulations = 120
# Load the coordinates DataFrame
coordinates_file = 'coordinates2.csv'  # Change this to the path of your coordinates CSV file
coordinates2_df = pd.read_csv(coordinates_file)


bkg = Background2D(data,(128,128),filter_size=(3,3))
thr = bkg.background + (1.6*bkg.background_rms)
segm = detect_sources(data, thr, npixels=10)

In [4]:
# Define the function to create segmentation maps
def create_segmentation_map(data):
    sigma_clip = SigmaClip(sigma=3.)
    bkg_estimator = Background2D(data, (128, 128), filter_size=(3, 3), sigma_clip=sigma_clip)
    bkg_rms = bkg_estimator.background_rms
    threshold = bkg_estimator.background + (1.6 * bkg_rms)
    segm = detect_sources(data, threshold, npixels=10)
    return segm.data


In [5]:
# Loop over each simulation folder
for index in range(num_simulations):
    simulation_dir = os.path.join(data_folder, f'simulation_{index}')
    fits_file = os.path.join(simulation_dir, f'sci_convolved_{index}.fits')
    
    # Load the simulated data
    hdul = fits.open(fits_file)
    data = hdul[0].data
    
    # Create the segmentation map
    segm = create_segmentation_map(data)
    
    # Save the segmentation map
    segm_file = os.path.join(simulation_dir, f'segmentation_map_{index}.fits')
    fits.writeto(segm_file, segm, overwrite=True)
    
    # Close the FITS file
    hdul.close()

In [6]:
def center_crop(data, center, size = 200):

    x_center, y_center = center
    y_min = y_center - (size // 2) 
    y_max = y_min + size
    
    x_min = x_center - (size // 2)
    x_max = x_min + size
    return data[y_min:y_max, x_min:x_max]
    


In [7]:
# Loop over each simulation folder
for index, row in coordinates2_df.iterrows():  # Use coordinates2_df for the correct column names
    simulation_dir = os.path.join(data_folder, f'simulation_{index}')
    
    # Load the segmentation map
    segm_file = os.path.join(simulation_dir, f'segmentation_map_{index}.fits')
    segm_hdul = fits.open(segm_file)
    segm_data = segm_hdul[0].data
    
    # Load the center coordinates
    center = (row['x0'], row['y0'])
    
    # Crop the segmentation map
    cropped_segm_data = center_crop(segm_data, center)
    
    # Save the cropped segmentation map
    cropped_segm_file = os.path.join(simulation_dir, f'cropped_segmentation_map_{index}.fits')
    fits.writeto(cropped_segm_file, cropped_segm_data, overwrite=True)
    
    # Load the science convolved file
    sci_convolved_file = os.path.join(simulation_dir, f'sci_convolved_{index}.fits')
    sci_convolved_hdul = fits.open(sci_convolved_file)
    sci_convolved_data = sci_convolved_hdul[0].data
    
    # Crop the science convolved file
    cropped_sci_convolved_data = center_crop(sci_convolved_data, center)
    
    # Save the cropped science convolved file
    cropped_sci_convolved_file = os.path.join(simulation_dir, f'cropped_sci_convolved_{index}.fits')
    fits.writeto(cropped_sci_convolved_file, cropped_sci_convolved_data, overwrite=True)
    
    # Close the FITS files
    segm_hdul.close()
    sci_convolved_hdul.close()


variance:


In [8]:
# Loop over each simulation folder
for index in range(num_simulations):
    simulation_dir = os.path.join(data_folder, f'simulation_{index}')
    print("Simulation Directory:", simulation_dir)
    
    # Load the simulated data
    fits_file = os.path.join(simulation_dir, f'sci_convolved_{index}.fits')
    hdul = fits.open(fits_file)
    data = hdul[0].data
    
    # Define sigma clipping parameters
    sigma_clip = SigmaClip(sigma=3.0)
    
    # Calculate background RMS
    bkg_estimator = StdBackgroundRMS()
    bkg = Background2D(data, (50, 50), filter_size=(3, 3),
                       sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
    
    # Extract the variance image
    bkg_rms = bkg.background_rms
    
    # Save the background RMS data as a FITS file
    bkg_rms_file = os.path.join(simulation_dir, f'variance_bkg_{index}.fits')
    fits.writeto(bkg_rms_file, bkg_rms, overwrite=True)
    
    # Close the FITS file
    hdul.close()


Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_0
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_1
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_2
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_3
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_4
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_5
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_6
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_7
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_8
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_9
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_10
Simulation Directory: /Users/nasekhian/Documents/GitHub/IMcascade/simulation_11
Simulation Directory: /Users/nasekhian/Documents/G

In [9]:
# Loop over each simulation folder
for index, row in coordinates2_df.iterrows():  # Use coordinates2_df for the correct column names
    simulation_dir = os.path.join(data_folder, f'simulation_{index}')
    
    # Load the segmentation map
    variance_file = os.path.join(simulation_dir, f'variance_bkg_{index}.fits')
    var_hdul = fits.open(variance_file)
    var_data = var_hdul[0].data
    
    # Load the center coordinates
    center = (row['x0'], row['y0'])
    
    # Crop the segmentation map
    cropped_var_data = center_crop(var_data, center)
    
    # Save the cropped segmentation map
    cropped_var_file = os.path.join(simulation_dir, f'cropped_var_{index}.fits')
    fits.writeto(cropped_var_file, cropped_var_data, overwrite=True)
    
    # Close the FITS files
    var_hdul.close()
